## Análisis de emanación de CO2 a nivel mundial. 

In [9]:
pip install pyjanitor 

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install missingno

Note: you may need to restart the kernel to use updated packages.


In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import sklearn.metrics
import janitor
import missingno

In [12]:
df = pd.read_csv('../data/raw/production_based_co2_emissions.csv')

In [13]:
df.dtypes

ISO3                                      object
Country                                   object
Continent                                 object
Hemisphere                                object
Metric tons of CO2e per capita (1990)    float64
Metric tons of CO2e per capita (1995)    float64
Metric tons of CO2e per capita (2000)    float64
Metric tons of CO2e per capita (2005)    float64
Metric tons of CO2e per capita (2010)    float64
Metric tons of CO2e per capita (2013)    float64
Metric tons of CO2e per capita (2018)    float64
dtype: object

El df consta de 181 filas y 11 columnas. 

In [14]:
df.shape

(181, 11)

In [15]:
df.head(5)

,ISO3,Country,Continent,Hemisphere,Metric tons of CO2e per capita (1990),Metric tons of CO2e per capita (1995),Metric tons of CO2e per capita (2000),Metric tons of CO2e per capita (2005),Metric tons of CO2e per capita (2010),Metric tons of CO2e per capita (2013),Metric tons of CO2e per capita (2018)
0,AFG,Afghanistan,Asia,Northern Hemisphere,1.24,0.84,0.82,0.76,1.06,1.40,2.66
1,ALB,Albania,Europe,Northern Hemisphere,3.56,2.15,2.36,2.68,2.83,2.85,3.52
2,DZA,Algeria,Africa,Northern Hemisphere,3.50,3.42,3.75,4.04,4.58,4.88,5.18
3,AGO,Angola,Africa,Southern Hemisphere,4.29,4.43,4.20,5.06,6.32,5.99,2.59
4,ATG,Antigua and Barbuda,America,Northern Hemisphere,6.31,6.53,7.08,8.77,10.69,11.22,12.59


análisis de nulos: El año 2010 es la única columna que no tiene valores nulos. 

In [16]:
(
    df
    .isnull()
    .any()
)

ISO3                                     False
Country                                  False
Continent                                False
Hemisphere                               False
Metric tons of CO2e per capita (1990)     True
Metric tons of CO2e per capita (1995)     True
Metric tons of CO2e per capita (2000)     True
Metric tons of CO2e per capita (2005)     True
Metric tons of CO2e per capita (2010)    False
Metric tons of CO2e per capita (2013)     True
Metric tons of CO2e per capita (2018)     True
dtype: bool

Sumatoria de valores nulos por variable: En los 90 tenemos la mayor cantidad de valores nulos. 

In [17]:
(
    df
    .isnull()
    .sum()
    
)

ISO3                                      0
Country                                   0
Continent                                 0
Hemisphere                                0
Metric tons of CO2e per capita (1990)    27
Metric tons of CO2e per capita (1995)     4
Metric tons of CO2e per capita (2000)     2
Metric tons of CO2e per capita (2005)     2
Metric tons of CO2e per capita (2010)     0
Metric tons of CO2e per capita (2013)     1
Metric tons of CO2e per capita (2018)     1
dtype: int64